# Content and Objective

+ Show calculations of Levinson-Durbin
+ Method: Get random ar signal and apply equations of LD and YW, respectively

In [ ]:
# importing
import numpy as np
import scipy.signal
from numpy.typing import ArrayLike

import matplotlib.pyplot as plt
from shutil import which

# showing figures inline
%matplotlib inline

In [ ]:
# plotting options
plt.rcParams.update({
    "font.size": 36,
    "text.usetex": bool(which("latex")),
    "figure.figsize": (30, 8)
})

# Get Parameter by Applying YW

In [ ]:
def est_acf(y: ArrayLike, est_type: str) -> ArrayLike:
    """
    estimates the discrete autocorrelation function (vector) given an array of observation

    :param y observation
    :param est_type "biased" or "unbiased"
    :return estimated acf, centered around 0
    """

    N = np.size(y)
    r = np.zeros_like(y)

    # loop lags of acf
    for k in np.arange(0, N):

        temp = np.sum(y[k:N] * np.conjugate(y[0:(N - k)]))

        # type of estimator
        if est_type == 'biased':
            r[k] = temp / N
        elif est_type == 'unbiased':
            r[k] = temp / (N - k)

    # find values for negative indices
    r_reverse = np.conjugate(r[::-1])

    return np.append(r_reverse[0:len(r) - 1], r)


def find_yule_walker_parameters(y: ArrayLike, order: int,
                                sigma2: float) -> ArrayLike:
    """
    Estimates the a_v paramters using the Yule-Walker method

    :param y samples
    :param order Yule-Walker model order
    :returns array of Yule-Walker parameters
    """

    N = len(y)

    #r = est_acf(y, 'biased')
    r = np.correlate(y, y, 'full') / N

    # get matrix R for Yule-Walker
    # note that R is not the autocorrelation matrix, but R = (ACF matrix)^*
    R = np.zeros([order + 1, order + 1], dtype=float)
    for p in range(0, order + 1):
        R[:, p] = r[N - 1 - p:N - 1 - p + order + 1]

    # find and solve linear equation system for the coefficients
    b = np.matrix(np.append(sigma2, np.zeros(order))).T
    theta = np.linalg.solve(R, b)

    theta /= theta[0]

    return np.transpose(theta)

# Construct Signal

### Parameters

In [ ]:
# parameters: noise variance
sigma_x_2 = 1 + np.random.rand()
print(f'Noise variance: \t sigma2 = {sigma_x_2}')

# parameters: ar parameters
q = 2
a_1_q = np.random.rand(q)
a_1_q = np.array([.5, .25])
print(f'Feedback parameters: \t a = {a_1_q}')


### Sample noise and apply to signal

In [ ]:
def get_ar_signal(sigma_x_2: float, N_length: int,
                  taps: ArrayLike) -> ArrayLike:
    """
    Get a signal from an autoregressive (AR) signal model

    :param sigma_x_2 input white noise variance
    :param N_length length of the signal vector
    :param order Order of the AR model
    :param Taps of the AR model
    :return Signal
    """

    # init input noise and output as first input value
    x = np.sqrt(sigma_x_2) * np.random.randn(N_length)
    y = np.zeros_like(x)

    # loop for times
    for n, value in enumerate(x):
        ar = sum(taps[_q] * y[n - _q - 1]
                 for _q in range(min(n, len(taps))))
        # apply
        y[n] = value - ar

    return y

# Apply Levinson-Durbin

+ Init: 
$$\theta_1=-\frac{\rho_1}{\rho_0}, \quad k_1=\theta_1, \quad \sigma_1^2=\rho_0-\frac{|\rho_1|^2}{\rho_0}$$

+ Loop q = 1 : max:
$$ k_{q+1}=-\frac{ \rho_{q+1}+\mathbf{r}_{q,\leftarrow}\mathrm{\theta}_q }{\sigma_q^2}$$
$$\sigma_{q+1}^2 =\sigma_q^2 ( 1-|k_{q+1}|^2)$$
$$\mathbf{\theta}_{q+1} = \begin{pmatrix} \mathbf{\theta}_q \\ 0 \end{pmatrix} + k_{q+1}\begin{pmatrix} \mathbf{\theta}_{q, \leftarrow} \\ 1 \end{pmatrix}$$

In [ ]:
# max value of q
q_max = 15

# sequence length
N_len = int(1e3)

# get ar signal
y = get_ar_signal(sigma_x_2, N_len, a_1_q)

# estimate acf and get indices/shifts >= 0 only
r = est_acf(y, 'biased')
r_pos = r[(len(r) - 1) // 2:]

# init LD
theta = [-r_pos[1] / r_pos[0]]
k = theta
sigma2 = r_pos[0] - np.abs(r_pos[1])**2 / r_pos[0]

show_intermediate_results = True
show_final_results = True
if show_intermediate_results:
    print(f" {'_' * 70} ")
    print(f"| q |    k   |   𝜎²   |")
    print(f"|{' ' * 31}𝚯(YW){' ' * 31}|")
    print(f"|{' ' * 31}𝚯    {' ' * 31}|")
    print(f"|{'-' * 68}|")
# loop for orders
for _q in range(1, q_max + 1):

    # slice acf values and determine inverted values
    r_q = r_pos[1:_q + 1]
    r_q_left = r_q[::-1]

    # determine new k, sigma2 and theta
    k = -(r_pos[_q + 1] + np.inner(r_q_left, theta)) / sigma2

    sigma2 = sigma2 * (1 - np.abs(k)**2)

    theta = np.append(theta, 0) + k * np.append(theta[::-1], 1)

    # like to see intermediate results?

    if show_intermediate_results:
        print(f"|{_q:3}|{k: >8.4f}|{sigma2: >8.4g}|")
        print(f"|{theta}|")
        print(f'|{find_yule_walker_parameters(y, _q+1, sigma2 )[0][1:]}|')
        print(f"|{'-'*68}|")

# like to see final results?
if show_final_results:
    print('Final results:')
    print('--------------')
    print(f'k: \t\t{k}')
    print(f'sigma2: \t {sigma2}\n')
    print(f'theta: \t\t {theta}\n')
    print(
        f'YW: theta: \t {find_yule_walker_parameters(y, q_max+1, sigma2 )[0][1:]} \n'
    )


# Redefine LD as Function to Time it

In [ ]:
def find_parameters_levinsondurbin(y: ArrayLike, q_max: int) -> ArrayLike:
    """
    estimates a_v parameters of filter using the Levinson-Durbin method
                
    :param y signal
    :param q_max model order
    :return parameter vector (a_v)
    """

    r = est_acf(y, 'biased')
    r_pos = r[(len(r) - 1) // 2:]

    # init
    theta = [-r_pos[1] / r_pos[0]]
    k = theta
    sigma2 = r_pos[0] - np.abs(r_pos[1])**2 / r_pos[0]

    for _q in range(1, q_max + 1):

        r_q = r_pos[1:_q + 1]
        r_q_left = r_q[::-1]

        k = -(r_pos[_q + 1] + np.inner(r_q_left, theta)) / sigma2

        sigma2 = sigma2 * (1 - np.abs(k)**2)

        theta = np.append(theta, 0) + k * np.append(theta[::-1], 1)

    return theta


def find_parameters_yulewalker(acf: ArrayLike, order: int,
                               sigma2: float) -> ArrayLike:
    """
    estimates a_v parameters of filter using the Yule-Walkier method
                
    :param acf autocorrelation of signla
    :param order model order
    :param sigma2 noise variance
    :return parameter vector (a_v)
    """

    N = int((len(acf) + 1) / 2)

    # get matrix R for Yule-Walker
    # note that R is not the autocorrelation matrix, but R = (ACF matrix)^*
    R = np.zeros([order + 1, order + 1], dtype=float)
    for p in range(0, order + 1):
        R[:, p] = acf[N - 1 - p:N - 1 - p + order + 1]

    # find and solve linear equation system for the coefficients
    b = np.matrix(np.append(sigma2, np.zeros(order))).T
    theta = np.linalg.solve(R, b)
    theta /= theta[0]

    return np.transpose(theta)

### time it

In [ ]:
import time

# max value of q
q_max = 100

# sequence length
N_len = int(1e4)

# get ar signal
y = get_ar_signal(sigma_x_2, N_len, a_1_q)

r = est_acf(y, 'biased')
r_pos = r[(len(r) - 1) // 2:]

# do LD
start = time.time()
t = find_parameters_levinsondurbin(r_pos, q_max)
elapsed = time.time() - start

print(f'LD required: {elapsed}')

# do YW
start = time.time()
for _q in range(q_max + 1):
    t = find_parameters_yulewalker(r, q_max, sigma2)
elapsed = time.time() - start

print(f'YW required: {elapsed}')
